In [12]:
from braket.tracking import Tracker
from braket.aws import AwsDevice
import json
import numpy as np
from braket.ahs.atom_arrangement import AtomArrangement
#from ahs_utils import show_register
from braket.ahs.driving_field import DrivingField
#from ahs_utils import show_global_drive
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation
from braket.devices import LocalSimulator
#from ahs_utils import show_final_avg_density
import time
import os
import datetime

version = '1.1'
    
# Use Braket SDK Cost Tracking to estimate the cost to run this example
tracker = Tracker().start()

# Get device capabilities
qpu = AwsDevice("arn:aws:braket:us-east-1::device/qpu/quera/Aquila")
field_of_view_width = qpu.properties.paradigm.lattice.area.width
field_of_view_height = qpu.properties.paradigm.lattice.area.height
n_site_max = qpu.properties.paradigm.lattice.geometry.numberSitesMax
C6 = float(qpu.properties.paradigm.rydberg.c6Coefficient)

# define postprocessing
def get_counters_from_result(result):
    post_sequences = [list(measurement.post_sequence) for measurement in result.measurements]
    post_sequences = ["".join(['r' if site==0 else 'g' for site in post_sequence]) for post_sequence in post_sequences]
    counters = {}
    for post_sequence in post_sequences:
        if post_sequence in counters:
            counters[post_sequence] += 1
        else:
            counters[post_sequence] = 1
    return counters

def run_example(filename, Delta_final= 20e6, simulator_max_qubits=10, qpu_max_qubits=256, shots=100, nsteps=1000, plot=False):
    # start
    print('process', filename, datetime.datetime.now())
    timestamp_start=datetime.datetime.now().timestamp()

    # Find blockade radius
    blockade_radius = (C6/(Delta_final))**(1/6) # sqrt(delta^2 + omega^2)
    print(f'Delta_final     = {Delta_final}')
    print(f'C6              = {C6}')
    print(f'blockade_radius = {blockade_radius}')
    
    # load file
    with open(filename, 'r') as fh:
        instance_data = json.load(fh)
    
    # setup coordinates
    x_min = np.min([x for x,y in instance_data['description']['points'].values()])
    y_min = np.min([y for x,y in instance_data['description']['points'].values()])
    R = instance_data['description']['R']
    factor = (blockade_radius / R)
    data = np.zeros((len(instance_data['nodes']),2))
    for i,(x,y) in instance_data['description']['points'].items():
        data[int(i)-1,0] = (x-x_min)*factor
        data[int(i)-1,1] = (y-y_min)*factor
    print(f'data dimension: {data.shape}, factor: {factor}')

    # setup register
    register = AtomArrangement()
    for x, y in data:
        pos = (x, y)
        register.add(pos)

    # setup controls
    time_points = [0, 2.5e-7, 2.75e-6, 3e-6] # s
    amplitude_min = 0
    amplitude_max = 1.57e7  # rad / s
    detuning_min = -Delta_final  # rad / s
    detuning_max = Delta_final   # rad / s
    amplitude_values = [amplitude_min, amplitude_max, amplitude_max, amplitude_min]
    detuning_values = [detuning_min, detuning_min, detuning_max, detuning_max]
    phase_values = [0, 0, 0, 0]
    drive = DrivingField.from_lists(time_points, amplitude_values, detuning_values, phase_values)

    # build simulation
    ahs_program = AnalogHamiltonianSimulation(
        register=register, 
        hamiltonian=drive
    )
    ahs_program = ahs_program.discretize(qpu)
    
    # set engines
    num_qubits = len(instance_data['nodes'])
    print(f'num_qubits: {num_qubits}')
    run_simulator = simulator_max_qubits >= num_qubits
    run_qpu = qpu_max_qubits >= num_qubits
    
    # simulator
    try:
        if not run_simulator:
            raise ValueError('simulator turned off')
        print('run simulator', datetime.datetime.now())
        sim_timestamp = datetime.datetime.now().timestamp()
        sim = LocalSimulator("braket_ahs")
        start_time = time.time()
        sim_result = sim.run(ahs_program, shots=shots, nsteps=nsteps).result()
        end_time = time.time()
        sim_density = sim_result.get_avg_density()
        sim_counts = get_counters_from_result(sim_result)
        sim_dt = end_time-start_time
    except Exception as e:
        print('simulator', e)
        sim_result = None
        sim_timestamp = None
        sim_density = None
        sim_counts = None
        sim_dt = None
        
    # hardware
    try:
        if not run_qpu:
            raise ValueError('qpu turned off')
        print('run qpu', datetime.datetime.now())
        qpu_timestamp = datetime.datetime.now().timestamp()
        start_time = time.time()
        qpu_result = qpu.run(ahs_program, shots=100).result()
        end_time = time.time()
        qpu_density = qpu_result.get_avg_density()
        qpu_counts = get_counters_from_result(qpu_result)
        qpu_dt = end_time-start_time
    except Exception as e:
        print('qpu', e)
        qpu_result = None
        qpu_timestamp = None
        qpu_density = None
        qpu_counts = None
        qpu_dt = None
        
    # end
    timestamp_end=datetime.datetime.now().timestamp()        
        
    # save
    instance_name = os.path.split(os.path.splitext(filename)[0])[1]
    result_filename = f'results/result_{instance_name}_{timestamp_start}.json'
    result_data = dict(problem=dict(instance_name=instance_name, instance_data=data.tolist()),
                       meta=dict(timestamp_start=timestamp_start, timestamp_end=timestamp_end, version=version),
                       sim_result=dict(sim_density=sim_density.tolist() if sim_density is not None else None, sim_counts=sim_counts, sim_dt=sim_dt, sim_timestamp=sim_timestamp),
                       qpu_result=dict(qpu_density=qpu_density.tolist() if qpu_density is not None else None, qpu_counts=qpu_counts, qpu_dt=qpu_dt, qpu_timestamp=qpu_timestamp),
                       settings=dict(time_points = time_points, amplitude_values = amplitude_values, detuning_values = detuning_values,
                       phase_values = phase_values,
                       blockade_radius=float(blockade_radius), n_site_max=int(n_site_max), C6=float(C6), x_min=int(x_min), y_min=int(y_min), R=float(R), factor=float(factor)))
    with open(result_filename, 'w') as fh:
        json.dump(result_data, fh)
    print('saved to', result_filename)

    # show
    print('finished', datetime.datetime.now())
    if plot:
        display(show_register(register))
        display(show_global_drive(drive))
        if sim_result is not None:
            print('simulation')
            display(show_final_avg_density(sim_result))
        if qpu_result is not None:
            print('qpu')
            display(show_final_avg_density(qpu_result))
    
    return result_data

In [13]:
###

In [ ]:
import glob

for filename in glob.glob('instances/*.json'):
    result_data = run_example(filename)
    #break

process instances/UDMIS081EXT_N30_W1.0_R0.3.orig.json 2023-11-15 22:24:27.943514
Delta_final     = 20000000.0
C6              = 5.42e-24
blockade_radius = 8.04442268093887e-06
data dimension: (30, 2), factor: 4.348336584291281e-06
num_qubits: 30
simulator simulator turned off
run qpu 2023-11-15 22:24:27.948491
saved to results/result_UDMIS081EXT_N30_W1.0_R0.3.orig_1700087067.943621.json
finished 2023-11-15 22:25:28.134034
process instances/UDMIS028EXT_N90_W1.0_R0.1.orig.json 2023-11-15 22:25:28.134741
Delta_final     = 20000000.0
C6              = 5.42e-24
blockade_radius = 8.04442268093887e-06
data dimension: (90, 2), factor: 6.435538144751096e-06
num_qubits: 90
simulator simulator turned off
run qpu 2023-11-15 22:25:28.137938
saved to results/result_UDMIS028EXT_N90_W1.0_R0.1.orig_1700087128.134779.json
finished 2023-11-15 22:26:29.331389
process instances/UDMIS013EXT_N40_W1.0_R0.2.orig.json 2023-11-15 22:26:29.332119
Delta_final     = 20000000.0
C6              = 5.42e-24
blockade_ra